In [1]:
import numpy as np
import matplotlib 
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pandas import Series,DataFrame
%matplotlib inline
matplotlib.style.use('ggplot')
import sklearn

from datetime import datetime
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 8
print(np.version.full_version)
print(matplotlib.__version__)
print(pd.__version__)
print(sklearn.__version__)

2019/05/01 21:44:45
1.16.2
3.0.3
0.24.2
0.20.3


In [2]:
data_types = {
    'id': np.str,
    'click': np.bool_,
    'hour': np.str,
    'C1': np.uint16,
    'banner_pos': np.uint16,
    'site_id': np.object,
    'site_domain': np.object,
    'site_category': np.object,
    'app_id': np.object,
    'app_domain': np.object,
    'app_category': np.object,
    'device_id': np.object,
    'device_ip': np.object,
    'device_model': np.object,
    'device_type': np.uint16,
    'device_conn_type': np.uint16,
    'C14': np.uint16,
    'C15': np.uint16,
    'C16': np.uint16,
    'C17': np.uint16,
    'C18': np.uint16,
    'C19': np.uint16,
    'C20': np.uint16,
    'C21': np.uint16    
}

#features_mask_org =  ['C1', 'banner_pos', 'site_domain', 'site_category', 'app_domain', 'app_category', 'device_ip', 
#                 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 
#                 'C20', 'C21']
features_mask_org =  ['C1', 'banner_pos', 'site_domain', 'site_category', 'app_domain', 'app_category', 
                      'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
                     'C20', 'C21']

#features_mask = features_mask_org + ['day_week', 'hour_day']

target_mask = 'click'

## data reading
#date_parser = lambda x: pd.datetime.strptime(x, '%y%m%d%H')

# def derive_time_features(df, start_hour=None, remove_original_feature=False):
#     if start_hour is None:
#         start_hour = df['hour'][0]
        
#     df['hour_int'] = train_df['hour'].apply(lambda x: np.floor((x - start_hour) / np.timedelta64(1, 'h')).astype(np.uint16))
#     df['day_week'] = train_df['hour'].apply(lambda x: x.dayofweek)
#     df['hour_day'] = train_df['hour'].apply(lambda x: x.hour)
    
#     if remove_original_feature:
#         df.drop('hour', axis=1, inplace=True)
    
#     return df, start_hour


In [3]:
fp_train_f = "./avazu-ctr-prediction/train_f.csv"

fp_test_f = "./avazu-ctr-prediction/test_f.csv"

fp_lr_model = "./avazu-ctr-prediction/lr_model"
fp_train_oh = "./avazu-ctr-prediction/train_f_oh"
fp_train_col_counts = "./avazu-ctr-prediction/col_counts"

## submission files
fp_sub = "./avazu-ctr-prediction/LR_submission.csv"

fp_test = "./avazu-ctr-prediction/test.csv"
fp_test_f = "./avazu-ctr-prediction/test_f.csv"


In [4]:
import pickle  # to store temporary variable
cols_counts = pickle.load(open(fp_train_col_counts, 'rb'))

sum = 0
for k, v in cols_counts.items():
    sum +=  v.size
    print("{} : {}".format(k, v.size))

print("all features: {} ".format(sum))

C1 : 7
banner_pos : 7
site_domain : 7745
site_category : 26
app_domain : 559
app_category : 36
device_ip : 6729486
device_model : 8251
device_type : 5
device_conn_type : 4
C14 : 2626
C15 : 8
C16 : 9
C17 : 435
C18 : 4
C19 : 68
C20 : 172
C21 : 60
all features: 6749508 


In [5]:
## and set the rest to 'other'
k = 99
col_index = {}
for col in features_mask_org:
    col_index[col] = cols_counts[col][0: k].index

sum = 0
for k, v in col_index.items():
    sum +=  v.size
    print("{} : {}".format(k, v.size))

print("all features: {} ".format(sum))

C1 : 7
banner_pos : 7
site_domain : 99
site_category : 26
app_domain : 99
app_category : 36
device_model : 99
device_type : 5
device_conn_type : 4
C14 : 99
C15 : 8
C16 : 9
C17 : 99
C18 : 4
C19 : 68
C20 : 99
C21 : 60
all features: 828 


In [30]:
def rawcount(filename):
    f = open(filename, 'rb')
    lines = 0
    buf_size = 1024 * 1024
    read_f = f.raw.read

    buf = read_f(buf_size)
    while buf:
        lines += buf.count(b'\n')
        buf = read_f(buf_size)

    return lines
print(rawcount(fp_test))

4577465


In [31]:

import os
import gc
os.remove(fp_test_f) if os.path.exists(fp_test_f) else None


df_test_org = pd.read_csv(fp_test, chunksize = 1000000, iterator = True,
                          dtype=data_types,
                          
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )

print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
hd_flag = True  # add column names at 1-st row
sum = 0
for chunk in df_test_org:
    df = chunk.copy()
    
    del chunk
    gc.collect()
    
    sum +=  len(df.index)
    print("{} : {}".format(sum, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    
    for col in features_mask_org:
        df[col] = df[col].astype('object')
        # assign all the rare variables as 'other'
        df.loc[~df[col].isin(col_index[col]), col] = 'other'
    with open(fp_test_f, 'a') as f:
        df.to_csv(f, columns = features_mask_org + ["id"], header = hd_flag, index = False)
           
    del df
    
    hd_flag = False

gc.collect()
print("Over")


2019/05/01 22:18:27
1000000 : 2019/05/01 22:18:30
2000000 : 2019/05/01 22:18:46
3000000 : 2019/05/01 22:19:03
4000000 : 2019/05/01 22:19:19
4577464 : 2019/05/01 22:19:34
Over


In [32]:
print(rawcount(fp_test_f))

4577465


In [33]:
## data reading
df = pd.read_csv(fp_train_f, nrows = 6,
                 dtype={'id':str}, 
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )
df

,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,id,click
0,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,other,1,2,15706,320,50,1722,0,35,65535,79,1000009418151094273,False
1,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,711ee120,1,0,15704,320,50,1722,0,35,34548,79,10000169349117863715,False
2,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,8a4875bd,1,0,15704,320,50,1722,0,35,34548,79,10000371904215119486,False
3,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,6332421a,1,0,15706,320,50,1722,0,35,34548,79,10000640724480838376,False
4,1005,1,other,0569f928,7801e8d9,07d7df22,779d90c2,1,0,other,320,50,other,0,35,65535,157,10000679056417042096,False
5,1005,0,bb1ef334,f028772b,7801e8d9,07d7df22,8a4875bd,1,0,16920,320,50,1899,0,431,34541,117,10000720757801103869,False


In [34]:
## data reading
df = pd.read_csv(fp_test_f, nrows = 6,
                 dtype={'id':str}, 
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )
df

,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,id
0,1005,0,f6ebf28e,f028772b,7801e8d9,07d7df22,0eb711ec,1,0,8330,320,50,761,3,175,34539,23,10000174058809263569
1,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,ecb851b2,1,0,22676,320,50,2616,0,35,34547,51,10000182526920855428
2,1005,0,f3845767,28905ebd,7801e8d9,07d7df22,1f0bc64f,1,0,22676,320,50,2616,0,35,34547,51,10000554139829213984
3,1005,0,c4e18dd6,50e219e0,aefc06bd,0f2161f8,542422a7,1,0,other,320,50,1092,3,809,34620,61,10001094637809798845
4,1005,0,c4e18dd6,50e219e0,2347f47a,f95efa07,1f0bc64f,1,0,23160,320,50,2667,0,47,65535,221,10001377041558670745
5,1005,1,5b626596,f028772b,7801e8d9,07d7df22,68b6db2c,1,0,6563,320,50,572,2,39,65535,32,10001521204153353724


In [48]:
df = pd.read_csv(fp_test_f,  nrows = 10000,
                        dtype={'id':str}, 
                        index_col=None,
                        chunksize=1000000, iterator=True)     

print(col_index['C14'])
for chunk in df:
    for index, row in chunk.iterrows():
        for key, value in row.iteritems():
            #print("{} = {}".format(key,value))
            if key == "id":
                continue

            if key not in features_mask_org:
                print(index)
                print("{} = {}".format(key,value))
                raise Exception
            if value not in col_index[key]:
                print(index)
                print("{} = {}".format(key,value))
                raise Exception
            

UInt64Index([ 4687, 21611, 21189, 21191, 19771, 19772, 16208, 20108,  8330,
             19950, 15705, 15701, 15699, 15703, 15704, 15707, 15708, 15702,
             15706, 16615, 23804, 21767, 21768, 17239, 22676, 19251, 17654,
             17653,  6563, 21790, 16920, 20633, 23438, 17753, 17614, 23143,
             23144, 20634, 20251, 17264, 20093, 17262, 21769, 21770, 17014,
             22680, 22683, 22681, 23161, 20632, 23160, 21789, 17037, 16688,
             22682, 17016, 19665, 20277, 21153, 17017, 22268, 23866, 19016,
             17875, 20366, 19015, 17877, 20362, 19743, 22288,  9478, 20352,
             23221, 23222, 20508, 22267, 17894, 21893, 23224, 20346, 21665,
             23441, 20345, 23454, 17893, 22104, 21647, 23722, 21882, 23626,
             22813, 17747, 20312,  1037, 17081, 22254, 23642, 22261,   375],
            dtype='uint64')
0
C14 = 8330, <class 'str'>


Exception: 

In [25]:
import pickle  # to store temporary variable
lr_model = pickle.load(open(fp_lr_model, 'rb'))
oh_enc = pickle.load(open(fp_train_oh, 'rb'))


In [29]:
from sklearn.linear_model import SGDClassifier  # using SGDClassifier for training incrementally
from sklearn.metrics import log_loss

os.remove(fp_sub) if os.path.exists(fp_sub) else None

df_test_f = pd.read_csv(fp_test_f, 
                        dtype={'id':str}, 
                        index_col=None,
                        chunksize=1000000, iterator=True)      

hd = True
sum = 0
for chunk in df_test_f:
   
    print("{} : {}".format(sum, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    
    try:
        df_test = oh_enc.transform(chunk)
        
        sum +=  len(chunk.index)
                
        #----- predict -----#
        feature_test = df_test.columns.drop(['id'])
        test_X = df_test[feature_test]
        y_pred = lr_model.predict_proba(test_X)[:, 1]  # get the probability of class 1
        #----- generation of submission -----#
        chunk['click'] = y_pred
        with open(fp_sub, 'a') as f: 
            chunk.to_csv(f, columns=['id', 'click'], header=hd, index=False) 
        hd = False
        #print("end at {}".format(datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
        
        del chunk
        del df_test
        gc.collect()
    except:
        #print(sum)
        #print(chunk)
        
        del df_test_f
        raise
        
    
print(' - PY131 - ')


0 : 2019/05/01 22:06:31
100000 : 2019/05/01 22:06:56
200000 : 2019/05/01 22:07:30
300000 : 2019/05/01 22:08:04
400000 : 2019/05/01 22:08:36
500000 : 2019/05/01 22:09:05
600000 : 2019/05/01 22:09:37
700000 : 2019/05/01 22:10:07
800000 : 2019/05/01 22:10:41
900000 : 2019/05/01 22:11:15
1000000 : 2019/05/01 22:11:47
1100000 : 2019/05/01 22:12:17


C:\Users\qqinx\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1200000 : 2019/05/01 22:12:47
1300000 : 2019/05/01 22:13:17
1400000 : 2019/05/01 22:13:46
1500000 : 2019/05/01 22:14:16
1600000 : 2019/05/01 22:14:46
1700000 : 2019/05/01 22:15:17
1800000 : 2019/05/01 22:15:48


C:\Users\qqinx\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1900000 : 2019/05/01 22:16:17
2000000 : 2019/05/01 22:16:48
2000000
           C1  banner_pos site_domain site_category app_domain app_category  \
2000000  1005           0    c4e18dd6      50e219e0   2347f47a     8ded1f7a   
2000001  1005           0    bb1ef334      f028772b   7801e8d9     07d7df22   
2000002  1005           0    7687a86e      3e814130   7801e8d9     07d7df22   
2000003  1005           0    c4e18dd6      50e219e0   2347f47a     f95efa07   
2000004  1005           1    7e091613      f028772b   7801e8d9     07d7df22   
2000005  1005           1    16a36ef3      f028772b   7801e8d9     07d7df22   
2000006  1005           1    c4e18dd6      50e219e0   2347f47a     0f2161f8   
2000007  1002           0       other      50e219e0   7801e8d9     07d7df22   
2000008  1005           0    f3845767      f028772b   7801e8d9     07d7df22   
2000009  1005           0    f3845767      f028772b   7801e8d9     07d7df22   
2000010  1005           0    f3845767      f028772b   7801e8d9 

TypeError: invalid input format

In [ ]:
print(rawcount(fp_sub))